In [1]:
import fiftyone.zoo as foz
import fiftyone as fo
import fiftyone.brain as fob
import numpy as np
from pathlib import Path
import json

In [2]:
def load_json(path: Path):
    with path.open() as f:
        data = json.load(f)

    return data


def save_json(obj, path: Path):
    with path.open("w") as f:
        json.dump(obj, f)

In [3]:
dataset_name = "darrensohphotos"
dataset_dir = "../static/imgs/darrensohphotos"
dataset_type = fo.types.ImageDirectory

export_dir = "../output"

model_name = "clip-vit-base32-torch"

In [4]:
dataset = fo.Dataset.from_dir(
  dataset_dir=dataset_dir,
  dataset_type=dataset_type,
)

 100% |███████████████| 1894/1894 [563.7ms elapsed, 0s remaining, 3.8K samples/s]      


In [6]:
dataset

Name:        2023.01.11.01.46.36
Media type:  image
Num samples: 1894
Persistent:  False
Tags:        []
Sample fields:
    id:       fiftyone.core.fields.ObjectIdField
    filepath: fiftyone.core.fields.StringField
    tags:     fiftyone.core.fields.ListField(fiftyone.core.fields.StringField)
    metadata: fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.metadata.ImageMetadata)

In [7]:
dataset.head()

[<Sample: {
     'id': '63bda47da18537697d138ec1',
     'media_type': 'image',
     'filepath': '/Users/angyizhe/code/VIS/color-photo-viz/static/imgs/darrensohphotos/2013-09-21_03-10-00_UTC.jpg',
     'tags': [],
     'metadata': None,
 }>,
 <Sample: {
     'id': '63bda47da18537697d138ec2',
     'media_type': 'image',
     'filepath': '/Users/angyizhe/code/VIS/color-photo-viz/static/imgs/darrensohphotos/2013-09-21_04-55-40_UTC.jpg',
     'tags': [],
     'metadata': None,
 }>,
 <Sample: {
     'id': '63bda47da18537697d138ec3',
     'media_type': 'image',
     'filepath': '/Users/angyizhe/code/VIS/color-photo-viz/static/imgs/darrensohphotos/2013-09-21_12-54-59_UTC.jpg',
     'tags': [],
     'metadata': None,
 }>]

In [5]:
model = foz.load_zoo_model(model_name)

In [33]:
embeddings = dataset.compute_embeddings(model)

 100% |███████████████| 1894/1894 [6.2m elapsed, 0s remaining, 5.1 samples/s]      


In [38]:
np.save(f'{export_dir}/embeddings.npy', embeddings)

In [5]:
img_embeddings = np.load(f'{export_dir}/clip_embeddings.npy')
palette_embeddings = np.load(f'{export_dir}/palette_embeddings.npy')

In [31]:
# dataset.export(
#   export_dir=export_dir,
#   dataset_type=fo.types.FiftyOneDataset,
#   label_field=["embeddings"],
#   export_media=False
# )

Directory '../export' already exists; export will be merged with existing files
Exporting samples...
 100% |██████████████████| 1894/1894 [405.3ms elapsed, 0s remaining, 4.7K docs/s]      


In [6]:
img_2d_results = fob.compute_visualization(
  dataset,
  embeddings=img_embeddings,
  num_dims=2,
  verbose=False,
  seed=51,
)

img_3d_results = fob.compute_visualization(
  dataset,
  embeddings=img_embeddings,
  num_dims=3,
  verbose=False,
  seed=51,
)

palette_2d_results = fob.compute_visualization(
  dataset,
  embeddings=palette_embeddings,
  num_dims=2,
  verbose=False,
  seed=51,
)

palette_3d_results = fob.compute_visualization(
  dataset,
  embeddings=palette_embeddings,
  num_dims=3,
  verbose=False,
  seed=51,
)

Generating visualization...
Generating visualization...
Generating visualization...
Generating visualization...


In [7]:
output = []

for i in range(len(dataset)):
  # output.append({
  #   "img2d": img_2d_results.points[i].tolist(),
  #   "img3d": img_3d_results.points[i].tolist(),
  #   "palette2d": palette_2d_results.points[i].tolist(),
  #   "palette3d": palette_3d_results.points[i].tolist(),
  # })

  img_2d_x, img_2d_y = img_2d_results.points[i].tolist()
  img_3d_x, img_3d_y, img_3d_z = img_3d_results.points[i].tolist()

  palette_2d_x, palette_2d_y = palette_2d_results.points[i].tolist()
  palette_3d_x, palette_3d_y, palette_3d_z = palette_3d_results.points[i].tolist()

  output.append({
    "img2d": {
      "x": img_2d_x,
      "y": img_2d_y,
      "z": 0,
    },
    "img3d": {
      "x": img_3d_x,
      "y": img_3d_y,
      "z": img_3d_z,
    },
    "palette2d": {
      "x": palette_2d_x,
      "y": palette_2d_y,
      "z": 0,
    },
    "palette3d": {
      "x": palette_3d_x,
      "y": palette_3d_y,
      "z": palette_3d_z,
    },
  })

In [8]:
save_json(output, Path('../src/data/positions.json'))

In [31]:
session = fo.launch_app(dataset)

In [32]:
plot = img_2d_results.visualize()
plot.show()

session.plots.attach(plot)

FigureWidget({
    'data': [{'customdata': array(['63c294b5fbda3bdcbadfe22d', '63c294b5fbda3bdcbadfe22e',
    …